In [2]:
import torch
from torch.nn import Linear, functional as F, CrossEntropyLoss, Module
from torch.optim import SGD, Adam
from torch.utils.data import DataLoader, TensorDataset 

In [94]:
class LogisticRegression(Module):
    def __init__(self, in_features, num_classes=2, bias=True):
        super().__init__()
        self.linear = Linear(in_features, num_classes, bias)
        self.optimizer = Adam(self.parameters(), lr=1e-5)
        self.loss_fn = CrossEntropyLoss()
    
    def fit(self, train_loader, epochs = 100):
        for epoch in range(epochs):
            for x_value, y_value in train_loader:
                prediction = self.linear(x_value)
                y_labels = F.one_hot(y_value.long(),len(self.linear.bias)).float()
                loss = F.cross_entropy(prediction, y_labels)
                loss.backward()
                self.optimizer.zero_grad()
                self.optimizer.step()
                
            if epoch % 10 == 0:
                print(f'Epoch {epoch}: Loss {loss}')
    
    def score(self, val_loader):
        score = 0
        for x_value, y_value in val_loader:
            prediction = self.linear(x_value)
            values, indexes = torch.max(prediction, dim=1)
            for tensor in (indexes == y_value):
                score += torch.sum(tensor)
        return score / (len(val_loader) * val_loader.batch_size)
    
    def forward(self, X):
        return torch.argmax(self.linear(X))

In [95]:
import pandas as pd
import numpy as np

In [96]:
df = pd.read_csv('churn.csv')
df

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
0,0,1,0,1,0,1,0,0,2,0,0,0,0,0,1,2,29.85,0
1,0,0,0,34,1,0,0,2,0,2,0,0,0,1,0,3,56.95,0
2,0,0,0,2,1,0,0,2,2,0,0,0,0,0,1,3,53.85,1
3,0,0,0,45,0,1,0,2,0,2,2,0,0,1,0,0,42.30,0
4,0,0,0,2,1,0,1,0,0,0,0,0,0,0,1,2,70.70,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,1,1,24,1,2,0,2,0,2,2,2,2,1,1,3,84.80,0
7039,0,1,1,72,1,2,1,0,2,2,0,2,2,1,1,1,103.20,0
7040,0,1,1,11,0,1,0,2,0,0,0,0,0,0,1,2,29.60,0
7041,1,1,0,4,1,2,1,0,0,0,0,0,0,0,1,3,74.40,1


In [97]:
df.isnull().sum().sum()

0

In [98]:
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

In [99]:
X_ = torch.from_numpy(X).float()
y_ = torch.from_numpy(y).float()

In [100]:
from torch.utils.data import random_split

tensor_dataset = TensorDataset(X_, y_)
train_ds, val_ds = random_split(tensor_dataset, [5000, 2043])
len(train_ds), len(val_ds)

(5000, 2043)

In [101]:
batch_size = 128

train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size, shuffle=True)

In [102]:
model = LogisticRegression(17)

In [103]:
model.fit(train_loader, 100)

Epoch 0: Loss 7.2866692543029785
Epoch 10: Loss 1.9061838388442993
Epoch 20: Loss 2.929330825805664
Epoch 30: Loss 2.3971614837646484
Epoch 40: Loss 5.252129554748535
Epoch 50: Loss 3.5588302612304688
Epoch 60: Loss 4.951070785522461
Epoch 70: Loss 5.554623603820801
Epoch 80: Loss 2.908770799636841
Epoch 90: Loss 8.75461196899414


In [104]:
model.score(val_loader)

tensor(0.7407)

In [105]:
model(X_[0])

tensor(0)